In [1]:
from gurobipy import *
import pickle

In [53]:
rewards = pickle.load(open('/Users/ashton/school/cmsc828m/project/data/attacker_actions/20180411-rewards.pickle', 'rb'))

In [54]:
rewards[0][0][0]

{'bs': 2.1, 'cve_id': u'CVE-2016-6494', 'es': 3.9, 'is': 2.9}

In [152]:
NUM_ATTACKERS = 3
NUM_NODES = 100
NUM_SERVICES = 4
M = 99999
LOSS_AVERSION_FACTOR = 0.1
ATTACKER_LOSS_FACTOR = 0.3
# reward[s][theta][a] is reward when service s is used for attacker theta action a
# first attacker has 2 actions for service 0 and 3 for service 1

# defender rewards for actions on honeypots are negative of legit rewards for defenders
# attacker rewards for actions on legit servers are negative of legit rewards for defenders
# attacker rewards for actions on honeypots are the same as legit rewards for defenders

attacker_prob = [0.5, 0.4, 0.1]

#required services (must have 2 of service 0 and 3 of service 1)
x = [0.1, 0.08, 0.06, 0.04]

m = Model('0407')

x_prime = []
# mixed strategy over service options
for i in range(NUM_SERVICES):
    x_prime.append(m.addVar(lb=0, vtype=GRB.CONTINUOUS, name='x' + str(i)))

# n: attacker pure strategy
# n[theta][s][a] is whether attack a for service s for attacker theta is selected
# attackers only choose one attack
n = [] 
for theta in range(NUM_ATTACKERS):
    n_t = []
    for s in range(NUM_SERVICES):
        n_s = []
        for a in range(len(rewards[s][theta])):
            n_s.append(m.addVar(vtype=GRB.BINARY, name='n_{0}_{1}_{2}'.format(theta, s ,a)))
        n_t.append(n_s)
    n.append(n_t)
    
v = []
for v_i in range(NUM_ATTACKERS):
    v.append(m.addVar(vtype=GRB.CONTINUOUS, name='v_{0}'.format(v_i)))

m.setObjective(sum(attacker_prob[theta] * n[theta][s][a] * ((x[s] / (x[s] + x_prime[s])) * -1 * rewards[s][theta][a]['is'] + \
                            LOSS_AVERSION_FACTOR * (x_prime[s] / (x[s] + x_prime[s])) * rewards[s][theta][a]['is']) \
                    for s in range(NUM_SERVICES) \
                   for theta in range(NUM_ATTACKERS) \
                   for a in range(len(rewards[s][theta]))), GRB.MAXIMIZE)

#o = 0
#for s in range(NUM_SERVICES):
#    for theta in range(NUM_ATTACKERS):
#        for a in range(len(rewards[s][theta])):
#            o += attacker_prob[theta] * n[theta][s][a] * (x[s] * -1 * rewards[s][theta][a]['is'] + \
#                                                             x_prime[s] * rewards[s][theta][a]['is'])
            

# how to rewrite constraint that requires the attacker to choose the best action available given the 
# defender's best action

# I think we can restrict n to binary (instead of number of attacks) because attacking multiple times will just
# give a linear increase in the reward.  Also, I think the attacker can always just choose a single attack instead
# of multiple as there is going to always be a dominant single attack that will net the most points and the attacker
# will only pick multiple attacks if they have the same max value


m.addConstr(sum(x_prime[s] + x[s] for s in range(NUM_SERVICES)) == 1, 'defender_strat')
m.addConstrs((sum(sum(n[theta][s]) for s in range(NUM_SERVICES)) == 1 for theta in range(NUM_ATTACKERS)),
               'attacker_strat')
m.addConstrs((0 <= v[theta] - (rewards[s][theta][a]['bs'] * (x[s] / (x[s] + x_prime[s])) +
                                  -1 * ATTACKER_LOSS_FACTOR * rewards[s][theta][a]['es'] * (x_prime[s] / (x[s] + x_prime[s])))  \
                for s in range(NUM_SERVICES) \
             for theta in range(NUM_ATTACKERS) \
             for a in range(len(rewards[s][theta]))),  'attacker_best_strat_1')
m.addConstrs((v[theta] - (rewards[s][theta][a]['bs'] * (x[s] / (x[s] + x_prime[s])) +
                                  -1 * ATTACKER_LOSS_FACTOR * rewards[s][theta][a]['es'] * (x_prime[s] / (x[s] + x_prime[s])))  \
             <= (1 - n[theta][s][a]) * M
                for s in range(NUM_SERVICES)
             for theta in range(NUM_ATTACKERS) \
             for a in range(len(rewards[s][theta]))),  'attacker_best_strat_2')

m.optimize()



GurobiError: Divisor must be a constant

In [124]:
for var in m.getVars():
    print(var.varName, var.x)
print('Obj:', m.objVal)

('x0', 0.2165709230926782)
('x1', 0.1871422624626589)
('x2', 0.26247788531630756)
('x3', 0.053808929128355276)
('n_0_0_0', 0.0)
('n_0_2_0', 0.0)
('n_0_2_1', 0.0)
('n_0_3_0', 1.0)
('n_1_0_0', 0.0)
('n_1_0_1', 0.0)
('n_1_0_2', 0.0)
('n_1_2_0', 0.0)
('n_1_2_1', 0.0)
('n_1_2_2', 0.0)
('n_1_2_3', 0.0)
('n_1_2_4', 0.0)
('n_1_2_5', 0.0)
('n_1_2_6', 0.0)
('n_1_2_7', 0.0)
('n_1_2_8', 0.0)
('n_1_3_0', 0.0)
('n_1_3_1', 1.0)
('n_1_3_2', 0.0)
('n_2_0_0', 0.0)
('n_2_0_1', 0.0)
('n_2_0_2', 0.0)
('n_2_0_3', 0.0)
('n_2_1_0', 0.0)
('n_2_1_1', 0.0)
('n_2_1_2', 0.0)
('n_2_1_3', 5.689893001203927e-16)
('n_2_1_4', 0.0)
('n_2_1_5', 0.0)
('n_2_1_6', 0.0)
('n_2_1_7', 0.0)
('n_2_1_8', 0.0)
('n_2_1_9', 0.0)
('n_2_1_10', 0.0)
('n_2_1_11', 0.0)
('n_2_1_12', 0.0)
('n_2_1_13', 0.0)
('n_2_1_14', 0.0)
('n_2_1_15', 0.0)
('n_2_1_16', 0.0)
('n_2_1_17', 0.0)
('n_2_1_18', 0.0)
('n_2_1_19', 0.0)
('n_2_1_20', 0.0)
('n_2_1_21', 0.0)
('n_2_1_22', 0.0)
('n_2_1_23', 0.0)
('n_2_1_24', 0.0)
('n_2_1_25', 0.0)
('n_2_1_26', 0.0)
('n_

In [125]:
[(var.varName, var.x) for var in m.getVars() if var.x != 0]

[('x0', 0.2165709230926782),
 ('x1', 0.1871422624626589),
 ('x2', 0.26247788531630756),
 ('x3', 0.053808929128355276),
 ('n_0_3_0', 1.0),
 ('n_1_3_1', 1.0),
 ('n_2_1_3', 5.689893001203927e-16),
 ('n_2_3_2', 1.0),
 ('v_0', 0.12490087417992013),
 ('v_1', 0.13022978457757234),
 ('v_2', 0.23857321261202336)]

In [130]:
rewards[3][0][0], rewards[3][1][1], rewards[3][2][2]

({'bs': 5.1, 'cve_id': u'CVE-2014-0472', 'es': 4.9, 'is': 6.4},
 {'bs': 6.0, 'cve_id': u'CVE-2014-0482', 'es': 6.8, 'is': 6.4},
 {'bs': 10.0, 'cve_id': u'CVE-2014-0474', 'es': 10.0, 'is': 10.0})

In [136]:
m.getVarByName("x0")

0.2165709230926782

In [141]:
def baseline(x_prime_custom):
    return sum(attacker_prob[theta] * m.getVarByName("n_{0}_{1}_{2}".format(theta, s, a)).X * (x[s] * -1 * rewards[s][theta][a]['is'] + \
                                                            LOSS_AVERSION_FACTOR * m.getVarByName("x" + str(s)).X * rewards[s][theta][a]['is']) \
                    for s in range(NUM_SERVICES) \
                   for theta in range(NUM_ATTACKERS) \
                   for a in range(len(rewards[s][theta])))

In [144]:
NUM_ATTACKERS = 3
NUM_NODES = 100
NUM_SERVICES = 4
M = 99999
LOSS_AVERSION_FACTOR = 0.1
ATTACKER_LOSS_FACTOR = 0.3
# reward[s][theta][a] is reward when service s is used for attacker theta action a
# first attacker has 2 actions for service 0 and 3 for service 1

# defender rewards for actions on honeypots are negative of legit rewards for defenders
# attacker rewards for actions on legit servers are negative of legit rewards for defenders
# attacker rewards for actions on honeypots are the same as legit rewards for defenders

attacker_prob = [0.5, 0.4, 0.1]

#required services (must have 2 of service 0 and 3 of service 1)
x = [0.1, 0.08, 0.06, 0.04]

m = Model('0407')

x_prime = []
# mixed strategy over service options
for i in range(NUM_SERVICES):
    x_prime.append(m.addVar(lb=0, vtype=GRB.CONTINUOUS, name='x' + str(i)))

# n: attacker pure strategy
# n[theta][s][a] is whether attack a for service s for attacker theta is selected
# attackers only choose one attack
n = [] 
for theta in range(NUM_ATTACKERS):
    n_t = []
    for s in range(NUM_SERVICES):
        n_s = []
        for a in range(len(rewards[s][theta])):
            n_s.append(m.addVar(vtype=GRB.BINARY, name='n_{0}_{1}_{2}'.format(theta, s ,a)))
        n_t.append(n_s)
    n.append(n_t)
    
v = []
for v_i in range(NUM_ATTACKERS):
    v.append(m.addVar(vtype=GRB.CONTINUOUS, name='v_{0}'.format(v_i)))

m.setObjective(sum(attacker_prob[theta] * n[theta][s][a] * (x[s] * -1 * rewards[s][theta][a]['is'] + \
                                                            LOSS_AVERSION_FACTOR * x_prime[s] * rewards[s][theta][a]['is']) \
                    for s in range(NUM_SERVICES) \
                   for theta in range(NUM_ATTACKERS) \
                   for a in range(len(rewards[s][theta]))), GRB.MAXIMIZE)

#o = 0
#for s in range(NUM_SERVICES):
#    for theta in range(NUM_ATTACKERS):
#        for a in range(len(rewards[s][theta])):
#            o += attacker_prob[theta] * n[theta][s][a] * (x[s] * -1 * rewards[s][theta][a]['is'] + \
#                                                             x_prime[s] * rewards[s][theta][a]['is'])
            

# how to rewrite constraint that requires the attacker to choose the best action available given the 
# defender's best action

# I think we can restrict n to binary (instead of number of attacks) because attacking multiple times will just
# give a linear increase in the reward.  Also, I think the attacker can always just choose a single attack instead
# of multiple as there is going to always be a dominant single attack that will net the most points and the attacker
# will only pick multiple attacks if they have the same max value

m.addConstrs((x_prime[0] == x_prime[i] for i in range(len(x_prime))), "proportional")
m.addConstr(sum(x_prime[s] + x[s] for s in range(NUM_SERVICES)) == 1, 'defender_strat')
m.addConstrs((sum(sum(n[theta][s]) for s in range(NUM_SERVICES)) == 1 for theta in range(NUM_ATTACKERS)),
               'attacker_strat')
m.addConstrs((0 <= v[theta] - (rewards[s][theta][a]['bs'] * x[s] +
                                  -1 * ATTACKER_LOSS_FACTOR * rewards[s][theta][a]['es'] * x_prime[s]) \
                for s in range(NUM_SERVICES) \
             for theta in range(NUM_ATTACKERS) \
             for a in range(len(rewards[s][theta]))),  'attacker_best_strat_1')
m.addConstrs((v[theta] - (rewards[s][theta][a]['bs'] * x[s] +
                                  -1 * ATTACKER_LOSS_FACTOR * rewards[s][theta][a]['es'] * x_prime[s]) \
             <= (1 - n[theta][s][a]) * M
                for s in range(NUM_SERVICES)
             for theta in range(NUM_ATTACKERS) \
             for a in range(len(rewards[s][theta]))),  'attacker_best_strat_2')

m.optimize()



Optimize a model with 294 rows, 150 columns and 868 nonzeros
Model has 143 quadratic objective terms
Variable types: 7 continuous, 143 integer (143 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+05]
  Objective range  [1e-02, 3e-01]
  QObjective range [6e-02, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e-01, 1e+05]
Presolve removed 294 rows and 150 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 1 (of 4 available processors)

Solution count 1: -0.44 

Optimal solution found (tolerance 1.00e-04)
Best objective -4.400000000000e-01, best bound -4.400000000000e-01, gap 0.0000%


In [145]:
[(var.varName, var.x) for var in m.getVars() if var.x != 0]

[('x0', 0.18),
 ('x1', 0.18),
 ('x2', 0.18),
 ('x3', 0.18),
 ('n_0_2_1', 1.0),
 ('n_1_2_3', 1.0),
 ('n_2_1_14', 1.0),
 ('v_0', 0.2214),
 ('v_1', 0.22139999999487717),
 ('v_2', 0.26)]

In [147]:
m.objVal

-0.4400000000000001

In [6]:

m = Model("ashton")
a = m.addVar(vtype=GRB.CONTINUOUS, name="a", lb=3, ub=5)
m.setObjective(-1 * math.log(a), GRB.MAXIMIZE)
m.optimize()

TypeError: a float is required

In [4]:
import math